# Keyword-Only Arguments

## 术语约定

|原文|翻译|
|:-:|:-:|
|Keyword-Only Arguments|仅限关键字参数|
|parameter slots|参数槽|
|positional argument|位置参数|
|keyword argument|关键字参数|
|varargs|可变参数|
|keyword dictionary argument|关键字字典参数|
|regular argument|常规参数|

## 摘要

本PEP提议改变实参分配给形参的方式。尤其是，对于使用了“仅限关键字”声明的参数做出如下改变：该形参只能由关键字参数提供并且永远不会由位置参数自动填充。

## 出发点

在本PEP提出前，Python函数允许使用位置参数或者关键字参数的形式传入实参。对于可变数量的参数，Python使用可变参数句法（*name）接收这些参数，并将这些参数存入一个元组。

对于位置参数，可变参数通常在所有的参数槽均被填充时才会启用。

上述操作意味着，对于一个特定的形参，无论正确与否，若传入过多实参，该形参总是会按顺序被填充，多余的实参则存入可变参数。这种**默认操作**在一些情况中是不愿意看到的。

因此本PEP希望实现如下功能：对于一些特殊形参，除非使用关键字参数对其赋值，否则该形参永远不会被填充。

在本PEP提出前，实现上述功能（特定形参不被自动填充并且可变参数正常使用）的唯一做法是：

目标：不希望形参“A”被自动填充，同时可变参数又能正常工作

1. 从形参中删除“A”
2. 利用关键字字典参数接收传给“A”的实参
3. 从关键字字典参数中解析得到“A”

上述操作显然不够直接，因此本PEP提议添加“仅限关键字”声明，对于声明为“仅限关键字”的形参，位置参数不会自动填充这些形参

## 实际操作

1. 允许常规参数出现在可变参数的后面

```python
def func(*args, keyword_only_arg=None):
    ...
```

上述实现实际上很巧妙的实现了仅限关键字参数。若使用位置参数传参，按照顺序，所有多余的参数均会被*args接收，keyword_only_arg并不会被自动填充，除非主动使用关键字参数对keyword_only_arg参数进行填充。

值得注意的是，仅限关键字参数仅能通过关键字参数的形式进行填充，并且Python要求所有的形参都被填充。这导致，对于没有默认值的仅限关键字参数必须使用关键字参数句法对其填充，否则会报错

2. 允许省略可变参数的参数名

```python
def func(arg_1, arg_2, *, keyword_only_arg=None):
    ...
```

上述示例中可变参数没有参数名，这使得可变参数无法解析，因为没有形参去承接这些参数（类似于占位符）。这样的操作可以限制可变参数的解析，即无论传入多少实参，该函数仅会接受和处理指定的参数。

第2条旨在处理如下情况：一方面不希望使用可变参数，另一方面希望使用仅限关键字参数。

需要特别提出第2条的原因也很简单：

首先注意到第1条通过将关键字参数放置于可变参数后实现仅限关键字参数（不是通过特别声明或者标注），即可变参数的存在是仅限关键字参数生效的前提

若不希望使用可变参数（形参中不包含可变参数），那么按照第1条的实现方法，仅限关键字参数无法实现。因为此时没有“容器”承接多余的参数，若按照位置参数的形式进行传参，所有的形参均会被填充。

上述分析表明为了使仅限关键字参数真的起作用，可变参数不可或缺。这种情况下仅有两种方法可以实现一方面不希望使用可变参数，另一方面希望使用仅限关键字参数的目标：

* 使用可变参数，但是无论如何不会对传入的多余参数进行处理
* 使用可变参数并且接收多余参数，若检测到可变参数不为空，直接报错
```python
def func(arg_1, arg_2, *args, keyword_only_arg=None):
    if args: # 可变参数对应的元组不为空
        raise TypeError
```

第二条实现更复杂并且没有必要，不如直接对传入的可变参数进行处理。

上述分析最后确定了仅限关键字参数的用法：

1. 所有位于可变参数（*name）之后的形参（除了**kargs）均是仅限关键字参数
2. 允许使用单星号（*）表示不允许任何位置参数对该符号之后的形参进行填充（默认执行），并且若传入多余的参数，直接报错。

## 函数调用行为

在本PEP通过后，函数将会依照如下方式传参

1. 为每一个形参分配一个参数槽
2. 初始化所有参数槽，并标记为空"empty"
3. positional arguments首先被赋值，然后是keyword arguments
4. 对于每一个positional argument
    * 首先尝试按照顺序将传入的值绑定到标注为空的参数槽。若该参数槽不是一个varargs argument的参数槽（*描述的形参）则将该参数槽标记为"filled"
    * 若该参数槽是一个varargs argument，并且没有命名（单仅有一个 *），Raise Error
    * 否则将剩余的positional argument全部放入varargs argument对应的参数槽
5. 对于每一个keyword argument
    * 若该关键字是函数的一个形参，则直接将传入的值赋予该关键字对应的参数槽，并且将该参数槽标注为“filled”。若该关键字槽已经被标注为“filled”，则Raise Error
    * 否则，若存在"keyword dictionary"形参（**描述的形参），将传入的关键字记入该形参。若"keyword dictionary"形参中已经记录了该关键字，则直接Raise Error
    * 若与上述两种情况均不匹配，直接Raise Error
6. 最后
    * 对于varargs argument的参数槽，若没有数据填充该槽则直接将空列表作为该形参的值
    * 对于没有赋值的槽，若该槽有默认参数则使用默认参数填充该槽，否则Raise Error

根据本PEP中的描述，上述所有的Error均是TypeError（In accordance with the current Python implementation, any errors encountered will be signaled by raising TypeError. (If you want something different, that’s a subject for a different PEP.)），但是实测中对于重复传参的情况会raise SyntaxError并且无法被try捕捉，Python解释器直接抛出错误，这一情况将在下述示例中体现。

keyword-only argument实际上就发生在上述流程中的第4步。按照第4步的正常流程，所有的形参均会被按顺序赋值，然后才会将传入的多余positional argument计入varargs argument。但是有时候不希望这种默认的传参方式，而是希望有些参数一定要用关键字参数进行赋值并且不允许依照positional argument的方式对该形参进行赋值。

## 向后兼容

本PEP提出的方案是原有传参方式的超集，因此本PEP是向后兼容的。


## 示例

下述示例用于验证上述的传参流程。示例按照如下顺序展示：

1. 按照positional argument方式传参，并且没有仅限关键字参数和可变参数
2. 按照positional argument方式传参，没有可变参数接收多余参数
3. 按照positional argument方式传参，使用*
4. 按照positional argument方式传参，有可变参数接收多余参数
5. 按照positional argument方式传参，使用可变参数以及仅限关键字参数，仅限关键字参数使用默认值
6. 按照positional argument方式传参，仅限关键字参数没有默认值并且没有使用关键字参数赋值
7. 按照keyword argument方式对仅限关键字参数进行传参
8. 重复传参

In [21]:
# ---------------------- test_func_0 ---------------------
# 没有仅限关键字参数
# 没有可变参数
def test_func_0(arg_1, key_only_arg):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

# ---------------------- test_func_1 ---------------------
# 没有仅限关键字参数
# 有可变参数
def test_func_1(arg_1, key_only_arg, *args):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

    if args:
        print("args: ", end="")
        for i in args:
            print("{}\\".format(i), end=" ")
    else:
        print("")
    print("")

# ---------------------- test_func_2 ---------------------
# 有仅限关键字参数
# 有可变参数
# 仅限关键字参数没有默认值 
def test_func_2(arg_1, *args, key_only_arg):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

    if args:
        print("args: ", end="")
        for i in args:
            print("{}\\".format(i), end=" ")
    else:
        print("")
    print("")

# ---------------------- test_func_3 ---------------------
# 有仅限关键字参数
# 有可变参数，但是仅使用 * 
# 仅限关键字参数没有默认值 
def test_func_3(arg_1, *, key_only_arg):

    print("arg_1: {}".format(arg_1))
    print("key_only_arg: {}".format(key_only_arg))

# ---------------------- test_func_4 ---------------------
# 有仅限关键字参数
# 有可变参数
# 仅限关键字参数有默认值 
def test_func_4(arg_1, *args, key_only_arg=None):

    print("arg_1: {}".format(arg_1))

    if args:
        print("args: ", end="")
        for i in args:
            print("{}\\".format(i), end=" ")
    else:
        print("")
    print("")

    print("key_only_arg: {}".format(key_only_arg))

In [24]:
#  ------------------ 测试1，正常传参 ------------------ 
# 没有多余的实参传入
print("测试1：正常运行，步骤4第1种情况")
arg_1 = "test_func"
key_only_arg = "check"
test_func_0(arg_1, key_only_arg)

#  ------------------ 测试2，错误传参 ------------------ 
# 有多余的实参传入，
# 但是没有可变参数接收这些参数
print("\n测试2：错误运行，步骤4第1种情况")
arg_1 = "test_func"
key_only_arg = "check"
surplus_arg = "surplus arg"
try:
    test_func_0(arg_1, key_only_arg, surplus_arg)
except TypeError:
    print("test_func_0仅接收两个实参，但是传入了3个实参")

#  ------------------ 测试3，错误传参 ------------------ 
# 有多余的实参传入，
# 使用*表示不接收多余位置参数
print("\n测试3：错误运行，步骤4第2种情况")
arg_1 = "test_func"
key_only_arg = "check"
surplus_arg = "surplus arg"
try:
    test_func_3(arg_1, key_only_arg, surplus_arg)
except TypeError:
    print("test_func_0仅接收1个实参，但是传入了3个实参，*表示不接受任何多余的实参")

#  ------------------ 测试4，正确传参 ------------------ 
# 有多余的实参传入，
# 有可变参数接收这些参数
print("\n测试4：正确运行，步骤4第3种情况")
arg_1 = "test_func"
key_only_arg = "check"
surplus_arg = "surplus arg"
test_func_1(arg_1, key_only_arg, surplus_arg)

#  ------------------ 测试5，正确传参 ------------------ 
# 仅限关键字参数使用默认参数，
print("\n测试5：正确运行，步骤6第2种情况")
arg_1 = "test_func"
surplus_arg = "surplus arg"
test_func_4(arg_1, surplus_arg)

#  ------------------ 测试6，错误传参 ------------------ 
# 仅限关键字参数没有赋值，
print("\n测试6：错误运行，步骤6第2种情况")
arg_1 = "test_func"
surplus_arg = "surplus arg"
try:
    test_func_2(arg_1, surplus_arg)
except TypeError:
    print("仅限关键字参数没有被赋值")

# ------------------ 测试7，正确传参 ------------------ 
# 主动使用关键字参数对仅限关键字参数传参，
# 并且可变参数接收多余的位置参数
print("\n测试7：正确运行，仅限关键字参数的正确使用")
arg_1 = "test_func"
surplus_arg = "surplus arg"
key_only_arg = "check"
test_func_2(arg_1, surplus_arg, key_only_arg=key_only_arg)

测试1：正常运行，步骤4第1种情况
arg_1: test_func
key_only_arg: check

测试2：错误运行，步骤4第1种情况
test_func_0仅接收两个实参，但是传入了3个实参

测试3：错误运行，步骤4第2种情况
test_func_0仅接收1个实参，但是传入了3个实参，*表示不接受任何多余的实参

测试4：正确运行，步骤4第3种情况
arg_1: test_func
key_only_arg: check
args: surplus arg\ 

测试5：正确运行，步骤6第2种情况
arg_1: test_func
args: surplus arg\ 
key_only_arg: None

测试6：错误运行，步骤6第2种情况
仅限关键字参数没有被赋值

测试7：正确运行，仅限关键字参数的正确使用
arg_1: test_func
key_only_arg: check
args: surplus arg\ 


In [20]:
# ----------------- 重复传参 ----------------- 
# 按照positional argument的方式进行传参，
# key_only_arg按照关键字传参，但是重复赋值
# Python解释器直接抛出错误
print("\n错误运行，重复关键字传参")
arg_1 = "test_func"
key_only_arg = "check"
surplus_arg = "surplus arg"
try:
    test_func_2(arg_1, surplus_arg, 
    key_only_arg=key_only_arg, key_only_arg="check_2")
except SyntaxError:
    print("同一关键字重复传参")

SyntaxError: keyword argument repeated (Temp/ipykernel_14540/1502453310.py, line 10)